In [1]:
#tensorいじりのテストを行う
import torch
import torch.nn
x = torch.randn(10,10,10)
print(torch.cuda.is_available())
from audio_to_multiple_pose_gan.dataset import generate_batch, get_processor
from audio_to_multiple_pose_gan.config import get_config
import warnings
warnings.simplefilter('ignore')

True


In [2]:
import pandas as pd
from common.consts import AUDIO_SHAPE
batch_size = 32
train_csv: str = "~/Desktop/AI-Experiments/speechandtext2gesture/Gestures/train.csv"
#train_csv: str = "~/AI-Experiments/speechandtext2gesture/Gestures/train.csv"
#df = pd.read_csv(self.args.train_csv)
#cfg = get_config(self.args.config)
df = pd.read_csv(train_csv)
#cfg = get_config(config)
cfg: dict = {"processor": "audio_to_pose", "input_shape": [None, AUDIO_SHAPE]}
process_row, decode_pose = get_processor(cfg)

In [3]:
from common.consts import AUDIO_SHAPE
configs = {
    "audio_to_pose": {"num_keypoints": 98, "processor": "audio_to_pose", "flatten": False, "input_shape": [None, AUDIO_SHAPE]},
    "audio_to_pose_inference": {"num_keypoints": 98, "processor": "audio_to_pose_inference", "flatten": False, "input_shape": [None, AUDIO_SHAPE]}
}
cfg: dict = {"processor": "audio_to_pose", "input_shape": [None, AUDIO_SHAPE]}
process_row, decode_pose = get_processor(cfg)

In [4]:
row = df.sample(n=1).iloc[0]
X, Y = generate_batch(df, process_row, batch_size)

In [5]:
Y.shape

(32, 64, 98)

In [6]:
X.shape

(32, 67267)

In [7]:
df.sample(n=1).iloc[0]

dataset                                                    train
start                                            00:01:29.122456
end                                              00:01:33.326660
interval_id                                                 4230
pose_fn        Gestures/shelly/train/npz/4230-00:01:29.122456...
audio_fn       Gestures/shelly/train/audio/4230-00:01:23.7837...
video_fn       8._Plato_Part_III_-_Arguments_for_the_immortal...
speaker                                                   shelly
Name: 10347, dtype: object

In [8]:
from audio_to_multiple_pose_gan.static_model_factory import *
a2p = Audio2Pose(1, 98).to(device)
a2p(torch.tensor(X).float().to(device)).shape

torch.Size([32, 64, 98])

In [9]:
Y.shape

(32, 64, 98)

In [10]:
d_patchgan = D_patchgan(in_channels=64)

In [11]:
Y.shape

(32, 64, 98)

In [12]:
Y.shape

(32, 64, 98)

In [13]:
from audio_to_multiple_pose_gan.torch_layers import *
to_motion_delta(Y).shape

(32, 63, 98)

In [14]:
Y = torch.tensor(Y).float().to(device)

In [15]:
a2p = Audio2PoseGANS(1, 98).to(device)
pred_Y = a2p(torch.tensor(X).float().to(device))

torch.Size([32, 256, 64])


In [16]:
keypoints_regloss(Y, pred_Y, regloss_type="l1")

tensor(0.8784, device='cuda:0', grad_fn=<MeanBackward0>)

In [17]:
from audio_to_multiple_pose_gan.config import get_config
from audio_to_multiple_pose_gan.dataset import generate_batch, get_processor
#get_modelの実装は任せる
from audio_to_multiple_pose_gan.static_model_factory import Audio2PoseGANS, D_patchgan
from audio_to_multiple_pose_gan.torch_layers import to_motion_delta, keypoints_to_train, keypoints_regloss
from common.audio_lib import save_audio_sample
from common.audio_repr import raw_repr
from common.consts import RIGHT_BODY_KEYPOINTS, LEFT_BODY_KEYPOINTS, LEFT_HAND_KEYPOINTS, \
    RIGHT_HAND_KEYPOINTS, POSE_SAMPLE_SHAPE, G_SCOPE, D_SCOPE, SR, FRAMES_PER_SAMPLE, AUDIO_SHAPE
from common.evaluation import compute_pck
from common.pose_logic_lib import translate_keypoints, get_sample_output_by_config
from common.pose_plot_lib import save_side_by_side_video, save_video_from_audio_video
from torch import optim
import torch
import pandas as pd
import argparse
from torch import nn

def get_training_keypoints():
    training_keypoints = []
    training_keypoints.extend(RIGHT_BODY_KEYPOINTS)
    training_keypoints.extend(LEFT_BODY_KEYPOINTS)
    for i in range(5):
        training_keypoints.extend(RIGHT_HAND_KEYPOINTS(i))
        training_keypoints.extend(LEFT_HAND_KEYPOINTS(i))
    training_keypoints = sorted(list(set(training_keypoints)))
    return training_keypoints

In [18]:
poses = decode_pose(Y.cpu().numpy()[0], shift = [0,0], speaker="shelly")

In [19]:
G_model = Audio2PoseGANS(1, 98).to(device)

In [20]:
state_dict = torch.load("params/G_model_last.pth")

In [21]:
from collections import OrderedDict
param_dict = OrderedDict()
for key in state_dict:
    param_dict[key.replace("_orig_mod.","")] = state_dict[key]

In [22]:
G_model.load_state_dict(param_dict)

<All keys matched successfully>

In [25]:
G_model(torch.tensor(X).float().to(device))

torch.Size([32, 256, 64])


tensor([[[ 2.8013e-04, -9.5853e-01,  1.5811e-01,  ...,  1.2247e+00,
           1.2418e+00,  1.2486e+00],
         [ 1.4283e-04, -1.0165e+00,  1.3619e-01,  ...,  1.2192e+00,
           1.2536e+00,  1.2626e+00],
         [ 3.1919e-04, -1.0452e+00,  1.0584e-01,  ...,  1.2178e+00,
           1.2534e+00,  1.2720e+00],
         ...,
         [ 1.4552e-03, -9.1413e-01,  1.8575e-01,  ...,  1.2431e+00,
           1.2671e+00,  1.2820e+00],
         [ 6.6021e-04, -9.2148e-01,  1.5552e-01,  ...,  1.2378e+00,
           1.2629e+00,  1.2858e+00],
         [ 4.1131e-05, -9.4288e-01,  1.5790e-01,  ...,  1.2241e+00,
           1.2204e+00,  1.2515e+00]],

        [[ 2.9345e-04,  2.3178e-01,  6.8600e-02,  ...,  5.6841e-01,
           6.1496e-01,  6.2544e-01],
         [ 1.8441e-04,  1.7168e-01, -3.1800e-05,  ...,  5.7123e-01,
           6.3520e-01,  6.3237e-01],
         [ 2.9670e-04,  1.5457e-01, -2.3184e-02,  ...,  5.8094e-01,
           6.4753e-01,  6.4717e-01],
         ...,
         [ 1.6989e-04,  2